In [2]:
from pathlib import Path
from collections import defaultdict
import re

import numpy as np


p_lvl_header = r"DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST|  Elapsed time"
EXP_NAME = "rand_nn"
subject_id = "s038"

logs = Path(f"archive/log-no-seed/")
exp_metric = defaultdict(list)
exp_max_iters = {
    1: np.zeros(4, dtype=np.uint16),
    2: np.zeros(4, dtype=np.uint16),
    3: np.zeros(4, dtype=np.uint16),
}

for filename in logs.glob(f"{EXP_NAME}-{subject_id}-*-*.out"):
    _, _, exp_id, _ = filename.stem.split("-")
    txt = filename.read_text()

    log_metrics = list()
    log_max_iters = np.zeros(4, dtype=np.uint16)
    for i, level in enumerate(re.split(p_lvl_header, txt)[1:-1]):
        level_metrics = [
            l.split(",")[2].strip() for l in re.split(r"\n", level.strip("XX").strip())
        ]
        log_metrics.append(level_metrics)
        log_max_iters[i] = len(level_metrics)

    exp_metric[exp_id].append(log_metrics)
    np.maximum(
        exp_max_iters[len(exp_id)], log_max_iters, out=exp_max_iters[len(exp_id)]
    )
exp_max_iters

{1: array([84, 45, 42, 31], dtype=uint16),
 2: array([62, 42, 40, 34], dtype=uint16),
 3: array([55, 70, 50, 20], dtype=uint16)}

In [3]:
metric_stats = dict()
for exp_id, repetitions in exp_metric.items():
    rep_metrics = list()
    for repetition in repetitions:
        padded_rep = np.concatenate(
            [
                np.pad(
                    level,
                    (0, exp_max_iters[len(exp_id)][i] - len(level)),
                    mode="constant",
                    constant_values=np.nan,
                ).astype(np.float64)
                for i, level in enumerate(repetition)
            ],
        )
        rep_metrics.append(padded_rep)

    stacked_rep = np.stack(rep_metrics)
    metric_stats[exp_id] = {
        "mean": np.nanmean(stacked_rep, axis=0),
        "std": np.nanstd(stacked_rep, axis=0),
        "upper": np.nanmax(stacked_rep, axis=0),
        "lower": np.nanmin(stacked_rep, axis=0),
    }

/tmp/ipykernel_180062/1788591866.py:20: RuntimeWarning:

Mean of empty slice

/home/mathdugre/papers/fast-interpolate/.venv/fast-interpolate/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning:

Degrees of freedom <= 0 for slice.

/tmp/ipykernel_180062/1788591866.py:22: RuntimeWarning:

All-NaN slice encountered

/tmp/ipykernel_180062/1788591866.py:23: RuntimeWarning:

All-NaN slice encountered



In [31]:
import plotly.graph_objects as go
import plotly.express as px


figure_dir = Path("figures")
figure_dir.mkdir(exist_ok=True)

colors = px.colors.qualitative.Plotly
for stage in [1, 2, 3]:
    fig = go.Figure()

    for i, (exp_id, stats) in enumerate(metric_stats.items()):
        if len(exp_id) == stage:
            x = np.arange(len(stats["mean"]))

            color = colors[i % len(colors)]
            rgba_color = px.colors.hex_to_rgb(color) + (0.2,)

            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=stats["mean"],
                    mode="lines",
                    line=dict(color=color),
                    name=exp_id,
                    legendgroup=exp_id,
                )
            )

            fig.add_trace(
                go.Scatter(
                    x=np.concatenate([x, x[::-1]]),
                    y=np.concatenate([stats["upper"], stats["lower"][::-1]]),
                    fillcolor=f"rgba{rgba_color}",
                    line=dict(color=f"rgba{rgba_color}"),
                    legendgroup=exp_id,
                    showlegend=False,
                )
            )

    fig.update_layout(
        title=f"Metric value for stage {stage}",
        xaxis_title="X Axis",
        yaxis_title="Y Axis",
        legend_title="Lines",
        yaxis=dict(fixedrange=True)
    )

    fig.show()
    fig.write_html(figure_dir / f"metric_value-stage_{stage}.html")
    fig.write_image(figure_dir / f"metric_value-stage_{stage}.png")